### Install dependenies

In [ ]:
import os
import sys

root = os.path.split(os.getcwd())[0]
if root not in sys.path:
    sys.path.append(root)

In [ ]:
#install dependencies
!{sys.executable} -m pip install numpy pandas intervaltree matplotlib

In [ ]:
from datetime import datetime
from typing import Callable

In [ ]:
from strategy import PoolData, Pool, Token, Fee, Frequency
from strategy import Position, AbstractStrategy
from strategy import Backtest

### Prepare data

In [ ]:
pool = Pool(Token.WBTC, Token.WETH, Fee.MIDDLE)
data = PoolData.from_pool(pool, Frequency.DAY)
data.plot()

### Define strategy

In [ ]:
class ConstantStrategy(AbstractStrategy):        
    def rebalance(
        self,
        t: datetime,
        c: float,
        vol: float,
        l: Callable[[float], float],
        pool_data: PoolData,
    ) -> bool:
        if not self.portfolio.position("main"):
            self.portfolio.add_position(Position(id="main", a = c / 2, b = c * 2))
            pos = self.portfolio.position("main")
            pos.deposit(c, 1 / 2 / c, 1 / 2, pool_data.pool.fee.percent)

        return False

### Backtest

In [ ]:
backtest = Backtest(ConstantStrategy)
backtest.run(data, 0.01)
backtest.plot()